In [1]:
#作ったモデルを動かしてみる
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()


/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path="../../models/hf"
model_path="../../models/hf/0401_270m_fin"
model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)


/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, max_new_tokens=200, repetition_penalty=1.5)

In [5]:
import time
text_list=["今日はいい",
"富士山は",
"質問: 今日の天気は? 回答:",
"批判的な",
"大規模言語モデルは",
"AI研究の問題点は",
"化学研究の問題点は",
"I have a",
"Functional polymers are",
"機能性高分子は",           
"ホンダ フィット 販売",
]
text_list=[
"英語: He is a good man. 日本語: ",

]

for text in text_list:
    perp=perplexity(model,tokenizer,text)
    s_time=time.time()
    res=pipe(text)[0]["generated_text"]
    consumed_time=time.time()-s_time
    print("-------")
    print("input: ", text)
    print("perplexity: ",perp)
    print("time: ", consumed_time)
    print("time/character: ", consumed_time/len(res))
    print("output: ",res)

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------
input:  英語: He is a good man. 日本語: 
perplexity:  45.608665466308594
time:  4.977976560592651
time/character:  0.008553224330915208
output:  英語: He is a good man. 日本語: 彼は素晴らしい人です。
 「He's the one who made me feel like I was in love with him/But he doesn’t know how to tell you"(日本語詞) - "He knows that she will never be able or willingly accept his feelings, but it makes her wanting for someone else more difficult than ever before (Japanese lyrics).」
 「He has been waiting on my side all these years and now when they say no / But we can see what happens next...」(日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞・日本語詞
